In [1]:
from Dataset import lerCSV,lerXLSX
import warnings
import sys 
import numpy as np
import pickle
import theanets
from sklearn import preprocessing
from random import shuffle
from sklearn.metrics import precision_score,recall_score,log_loss
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from math import sqrt
import math
from roc import RocAucScoreOp
%matplotlib inline

In [2]:
warnings.filterwarnings("ignore")   
matrix = lerXLSX("Banco de Dados - Infarto treinoTeste.xlsx",training=True,agrupamento=False)
def sigmoid(x):
  x = (x - 0.5) *  10
  return 1 / (1 + math.exp(-x))

In [3]:
k = 10
lista = list(range(0,len(matrix))) 
shuffle(lista)
a = np.asarray(lista) % k

In [4]:
min_max_scaler = preprocessing.MinMaxScaler()
matrixProcessed = min_max_scaler.fit_transform(matrix[:,:3])
#matrixProcessed = preprocessing.normalize(matrix[:,:3], norm='l2')
#matrixProcessed = np.array([[matrixProcessed[y,0],matrixProcessed[y,1],matrixProcessed[y,2],matrixProcessed[y,3]]  for y in range(0,len(matrix[:,0]))])
matrixProcessed = matrixProcessed = np.array([[matrixProcessed[y,0],sigmoid(matrixProcessed[y,1]),matrixProcessed[y,2]]  for y in range(0,len(matrix[:,0]))])


In [11]:
def specificity(outputs,predicts):
    neg = np.sum([output == 0 for output in outputs])
    tn = np.sum([predicts[x] == outputs[x] and outputs[x] == 0 for x in range(len(predicts))])
    return tn / neg

def KFold(matrix,metric):
    k = 8
    
    limit = 3
    
    
    lista = list(range(1,len(matrix))) 
    shuffle(lista)
    a = np.asarray(lista) % k   
    
    scores = np.zeros(k)
    scoresT = np.zeros(k)
    recall = np.zeros(k)
    precision = np.zeros(k)
    average_precision = np.zeros(k)
    
    for fold in range(0,k):
        treinoIndex = np.where(a != fold)[0]
        testeIndex =  np.where(a == fold)[0]        
        
        inputs = np.array([[matrix[y,0],matrix[y,1],matrix[y,2]]  for y in treinoIndex])
        weights = np.array([int(matrix[y,3] == 1)*0+1 for y in treinoIndex],dtype="int32")        
        outputs = np.array([int(matrix[y,3] == 1) for y in treinoIndex],dtype="int32")
            
        model = theanets.Classifier(metric[1],loss='CrossEntropy',weighted=True)  
        saidaT = model.train([inputs, outputs,weights],algo='sgd',learning_rate=0.01,momentum=0.9)       
       
        scoresT[fold] = saidaT[0]["err"]
        testOutputs = [matrix[y,3] for y in testeIndex] 
        testInputs = np.array([matrix[y,:3]  for y in testeIndex])   
        predicts = model.predict(testInputs)
        predictsProbas = model.predict_proba(testInputs)
        scores[fold] = log_loss(testOutputs, predictsProbas)
         
        recall[fold] = recall_score(testOutputs, predicts)
        precision[fold] = specificity(testOutputs, predicts)        
    
    print("Final:"+ metric[0] + " = Error Treino:%2.2e[+/- %2.2e]"%(np.mean(scoresT),np.std(scoresT)))
    print("Final:"+ metric[0] + " = Error Validacao:%2.2e[+/- %2.2e]"%(np.mean(scores),np.std(scores)))
    print("Final:"+ metric[0] + " = Error:%2.2e[+/- %2.2e]"%(np.mean(scores),np.std(scores)))
    print("Final:"+ metric[0] + " = Sensitive:%2.2e[+/- %2.2e]"%(np.mean(recall),np.std(recall)))
    print("Final:"+ metric[0] + " = specificity:%2.2e[+/- %2.2e]"%(np.mean(precision),np.std(precision)))       
    return np.mean(scoresT),np.mean(scores)

In [12]:
inputOutput = np.array([np.concatenate((matrixProcessed[x,:3],[matrix[x,3] == 1]),axis=0) for x in range(0,len(matrix[:,3]))])

In [13]:
metrics = [   ["neural network (3,2)",[3, 2]],["neural network (3,1 tanh,2)",[3, (1,"tanh"), 2]],["neural network (3,2 tanh,2)",[3, (2, 'tanh'), 2]],["neural network (3,3 tanh,2)",[3, (3, 'tanh'), 2]],["neural network (3,5 tanh,2)",[3, (5, 'tanh'), 2]], ["neural network (3,7 tanh,2)",[3, (7, 'tanh'), 2]], ["neural network (3,10 tanh,2)",[3, (10, 'tanh'), 2]], ["neural network (3,15 tanh,2)",[3, (15, 'tanh'), 2]], ["neural network (3,5 tanh,5 tanh,2)",[3,(5, 'tanh'), (5, 'tanh'), 2]], ["neural network (3,10 tanh,5 tanh,2)",[3,(10, 'tanh'), (10, 'tanh'), 2]]]
saida = np.array([ KFold(inputOutput, metric=metric) for metric in metrics])
np.savetxt('3features.csv', saida, delimiter=',')
    


(OrderedDict([('loss', 0.50712110107747566), ('err', 0.50712110107747566), ('acc', 0.75189393939393934)]), OrderedDict([('loss', 0.50611043339095285), ('err', 0.50611043339095285), ('acc', 0.75189393939393945)]))
(OrderedDict([('loss', 0.49456081876804253), ('err', 0.49456081876804253), ('acc', 0.76927759740259738)]), OrderedDict([('loss', 0.49433998406899415), ('err', 0.49433998406899415), ('acc', 0.76643668831168832)]))
(OrderedDict([('loss', 0.48533297768682426), ('err', 0.48533297768682426), ('acc', 0.77623106060606062)]), OrderedDict([('loss', 0.48443367791933406), ('err', 0.48443367791933406), ('acc', 0.77481060606060614)]))
(OrderedDict([('loss', 0.48456971870721743), ('err', 0.48456971870721743), ('acc', 0.77481060606060614)]), OrderedDict([('loss', 0.48381349374973864), ('err', 0.48381349374973864), ('acc', 0.78049242424242427)]))
(OrderedDict([('loss', 0.48762277461286452), ('err', 0.48762277461286452), ('acc', 0.76912878787878791)]), OrderedDict([('loss', 0.48735151954666445